In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import sklearn.metrics as metrics
import heapq
import sklearn
import matplotlib.pyplot as plt

In [2]:
import joblib
import pickle
import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
remvwords = stopwords.words('english')
# newdata=pd.read_csv('./data/manualdata_remvname.csv')
qsword=['not','in','t']
# qsword=['what','who','when','why','where','whom','which','how','do','does','can','could','will','is','am','were','was','are','have','has','had',
#        'did','if','most','don','don\'t','should']
remv = ['the','these','it','a','an','them','to']
re_words = [x for x in remvwords if x not in qsword]
re_words.append('mendeley')
cos = metrics.pairwise.cosine_similarity

In [3]:
## cosine similarity
edu_qso = pd.read_csv('./data/newcla2ans.csv')
edu_qso=edu_qso[['title','link']]
edu_qso = edu_qso.iloc[:,:]
edu_qs=edu_qso
# edu_vect = TfidfVectorizer(ngram_range=(1,3),stop_words=re_words).fit(edu_qs['title'])
# edu_data = edu_vect.transform(edu_qs['title'])
# cla_2_qs = newdata.loc[newdata['label']==2]
# cla_2_qs = edu_vect.transform(cla_2_qs['qs'])
# def cosQA(query):
#     qs_vec=edu_vect.transform([query])
#     x = [cos(qs_vec,edu_data[i]) for i in range(0,303)]
#     index=x.index(max(x))
#     return edu_qs[index]['title']
# edu_qs.to_csv('./data/edu_answer.csv',index=False)

In [4]:
for_ans=pd.read_csv('./data/newcla2ans.csv')

In [5]:
lemmatizer=WordNetLemmatizer()
tokenizer = nltk.word_tokenize

In [6]:
def cosQA(query):
#     query = tokenizer(query)
#     query = [lemmatizer.lemmatize(x) for x in query]
#     query = ''.join(query)
    qs_vec=edu_vect.transform([query])
    x = [cos(qs_vec,edu_data[i]) for i in range(len(edu_qs['title']))]
    index=x.index(max(x))
    return edu_qs[index]['title']

In [7]:
def cleanText(query):
    query = query.lower()
    query = tokenizer(query)
    query = [word for word in query if word not in re_words]
    query = [word for word in query if word.isalpha()]
    query = [lemmatizer.lemmatize(x) for x in query]
    query = ' '.join(query)
    return query


In [8]:
for i in range(len(edu_qs['title'])):
    edu_qs['title'][i] = cleanText(edu_qs['title'][i])
edu_vect = TfidfVectorizer(ngram_range=(1,2),stop_words=re_words).fit(edu_qs['title'])
edu_data = edu_vect.transform(edu_qs['title'])

In [ ]:
pickle.dump(edu_vect,open('./question_classifier/type2ans_vect.pickle','wb'))

In [ ]:
vect_2ans = edu_vect
def type2ans(query):
    query = query.lower()
    query = nltk.tokenize.word_tokenize(query)
    query = [word for word in query if word.isalpha()]
    query = [lemmatizer.lemmatize(x) for x in query]
    query = ' '.join(query)
    v = vect_2ans.transform([query])
    x = [float(cos(v,edu_data[i])) for i in range(len(edu_qs['title']))]
    if any(x):
        index = np.argpartition(x,[-1,-2,-3,-4,-5])[-5:]
        index = index[::-1]
        print(index)
        print(x[index[0]])
        print(x[index[1]])
        print(x[index[2]])
        print(x[index[3]])
        print(x[index[4]])
        res_ind = 0
        for i in index:
            res_ind +=1
            ans=np.array(edu_qso.iloc[[i]]).tolist()
            ans = ans[0]
            title = ans[0]
            link = ans[1]
            # title = edu_qso.iloc[[i]][0]
            # link = edu_qso.iloc[[i]][1]
            print(res_ind,title,link)
    else:
        st.write('result','no answer found')

In [ ]:
query = 'ios'
# query = tokenizer(query)
# query = [lemmatizer.lemmatize(x) for x in query]
# query = ' '.join(query)
query = cleanText(query)
v = edu_vect.transform([query])
x = [float(cos(v,edu_data[i])) for i in range(len(edu_qs['title']))]
if any(x):
#     for i in heapq.nlargest(3,x):
#         index.append(x.index(i))
    index = np.argpartition(x,[-1,-2,-3,-4,-5])[-5:]
    index = index[::-1]
    for i in index:
        print(np.array(edu_qso.iloc[[i]]).tolist())
else:
    print('no answer')

# Test

In [10]:
class2_test = pd.read_csv('./data/class2test.csv')

In [11]:
pd.set_option('display.max_colwidth',1000,'display.width',1000)

In [12]:
for i in range(len(class2_test['question'])):
    class2_test['question'][i] = cleanText(class2_test['question'][i])

In [ ]:
res_2=[]
score=[]
for i in class2_test['question']:
    query = i
    v = edu_vect.transform([i])
#     x = [cos(v,edu_data[i]) for i in range(len(edu_qs['title']))]
    x = [float(cos(v,edu_data[i])) for i in range(len(edu_qs['title']))]
#     if any(x):
#         index = np.argpartition(x,-5)[-5:]
#         index = index[::-1]
#         res_ind = 0
#         for i in index:
#             res_ind +=1
#             ans=np.array(edu_qso.iloc[[i]]).tolist()
#             ans = ans[0]
#             title = ans[0]
#             link = ans[1]
#             # title = edu_qso.iloc[[i]][0]
#             # link = edu_qso.iloc[[i]][1]
#             print(res_ind,title,link)
    if any(x):
#         index = [x.index(i) for i in heapq.nlargest(10, x)]
        index = np.argpartition(x,[-1,-2,-3,-4,-5])[-5:]
        index = index[::-1]
        index = [ind for ind in index if x[ind]>0.306]
        score.append([x[i] for i in index])
        ans=[]
#         single_query = []
#         single_query = pd.DataFrame(single_query)
        for i in index:
#             single_query = pd.concat([single_query,edu_qso.iloc[[i]]],axis=1)
            ans.append(for_ans.iloc[[i],1:].title)
#             ans.append(np.array(for_ans.iloc[[i]]).tolist()[0][1:])
        res_2.append(ans)
#         res_2 = pd.concat([res_2,single_query],axis=0)
    else:
        res_2.append('no answer')
        score.append(0)
#         i = 'no answer'
#         res_2 = pd.concat([res_2,pd.Series(i)],axis=0)

In [ ]:
# class_2_res = pd.concat([class2_test,pd.Series(res_2)],axis=1)
res_total = pd.DataFrame({'answer':res_2,'score':score})

In [ ]:
# c1 = cla2['answer'].str.split(',',expand=True)

In [ ]:
cla2_test = pd.read_csv('./data/class2test.csv')

In [ ]:
cla2=pd.DataFrame({'question':cla2_test['question'],'answer':res_total['answer'],'score':res_total['score']})
cla2.to_csv('./result/cla2ansthres.csv',header=None,index=None)

In [ ]:
score1=[x for x in score if x!=0]

In [ ]:
# answer amount/how many data points
noans=0
one=0
two=0
three=0
four=0
five=0
for i in score1:
    if len(i)==0:
        noans+=1
    elif len(i)==1:
        one +=1
    elif len(i)==2:
        two +=1
    elif len(i)==3:
        three+=1
    elif len(i)==4:
        four+=1
    else: 
        five+=1
print(noans, one,two,three,four,five)

# DCG

In [13]:
####DCG score
scores = pd.read_csv('./data/newdcg_correctorder.csv')

In [14]:
iscores=scores.iscore
scores = scores.score

In [ ]:
s=[]
si=[]
for i,j in zip(scores,iscores):
    s1=[]
    s2=[]
    for x in (0,2,4,6,8):
        s1.append(int(i[x]))
        s2.append(int(j[x]))
    si.append(s2)
    s.append(s1)

In [ ]:
total = 0
dcg_s=[]
for i in s:
    dcg = i[0]+(i[1]/np.log2(2))+(i[2]/np.log2(3))+(i[3]/np.log2(4))+(i[4]/np.log2(5))
    dcg_s.append(dcg)
    total += dcg

In [ ]:
### nDCG =DCG/IDCG
total = 0
dcg_s=[]
for i,j in zip(s,si):
    dcg = i[0]+(i[1]/np.log2(2))+(i[2]/np.log2(3))+(i[3]/np.log2(4))+(i[4]/np.log2(5))
    idcg = j[0]+(j[1]/np.log2(2))+(j[2]/np.log2(3))+(j[3]/np.log2(4))+(j[4]/np.log2(5))
    ndcg=dcg/idcg
    dcg_s.append(ndcg)
    total += ndcg

In [ ]:
total/100

In [ ]:
dcg_o=2+(2/np.log2(2))+(2/np.log2(3))+(2/np.log2(4))+(2/np.log2(5))
dcg_o

# Threshold

In [ ]:
#threshold
th=[]
for i,j in zip(score,s):
    if i == 0 :
        i=[0,0,0,0,0]
        for x in range(0,5):
            th.append([round(i[x],3),j[x]])
    else:
        for x in range(0,5):
            th.append([round(i[x],3),j[x]])

In [16]:
th = np.array(th).T

In [ ]:
thp=pd.DataFrame({'score':th[0],'dcg':th[1]})

In [ ]:
gs=[]
bs=[]
ss=[]
threshold = 0.0
for i in range(0,1000):
    x = thp[thp.score>threshold]['dcg'].value_counts() 
#     if len(x) == 3:
#         x = x
    gs.append([threshold,x[2]]) 
    try:
        x[0]
    except KeyError:
        bs.append([threshold,0])
    else:
        bs.append([threshold,x[0]])
    try:
        x[1]
    except KeyError:
        ss.append([threshold,0])
    else:
        ss.append([threshold,x[1]])
#     elif len(x) == 2:
#         gs.append([threshold,x[0]])
#         bs.append([threshold,0])
#     else:
#         gs.append([threshold,x[0]])
#         bs.append([threshold,0])

    threshold += 0.001

In [18]:
gs=np.array(gs).T
bs=np.array(bs).T
ss=np.array(ss).T

In [ ]:
x = np.arange(0.0,1.0,.001)
plt.figure(figsize=(20,10))
# plt.xticks(x)
plt.plot(x,gs[1],linewidth=2.5,label = 'Good Answer')
plt.plot(x,bs[1],'g',label = 'Bad Answer')
plt.plot(x,ss[1],'r',label = 'soso Answer')
plt.grid()
plt.legend()


In [ ]:
gp=[]
gr=[]
for i in range(0,1000):
    precision = (gs[1][i]+ss[1][i])/(gs[1][i]+ss[1][i]+bs[1][i])
    recall = (gs[1][i]+ss[1][i])/(gs[1][1]+ss[1][1])
    gp.append(precision)
    gr.append(recall)

In [ ]:
x = np.arange(0.0,1.0,.001)
plt.figure(figsize=(20,10))
# plt.xticks(x)
plt.plot(x,gp,linewidth=2.5,label = 'precision')
plt.plot(x,gr,'g',label = 'recall')